# Prompt Templates

---

Hello everyone! In this notebook, we will explore the concept of "Prompt Templates," which you will frequently encounter in LangChain projects.

The goal of this module is to master:
1. Prompt Templates
2. Output Parsers

## Load environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv()

## By default, load_dotenv() will assign environment variables into os.environ, like following code:
## See environment variables in .env file
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] =  os.getenv('LANGCHAIN_API_KEY')
# os.environ["OPENAPI_KEY"] =  os.getenv('OPENAI_KEY')

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

# Chat Prompt Template

In Module 1, "Language Model," we passed prompt as a string or a list of messages directly into the language model.

However, in most cases, the input to a language model is constructed from a combination of user input and application logic. This application logic typically takes the raw user input and transforms it into a list of messages that are ready to be passed to the language model.

**PromptTemplates** are a concept in LangChain designed to facilitate this transformation. They take in raw user input and return data (a prompt) that is ready to be passed into a language model.

Let's create a `PromptTemplate` that accepts a user variable called `country`

In [8]:
from langchain_core.prompts import ChatPromptTemplate   

template = "What is capital city of {country}?"

prompt_template = ChatPromptTemplate.from_template(template)

# .invoke method will genereate prompt ready to be passed to the language model
message_prompt = prompt_template.invoke({"country": "Italia"})
# Alternatively, you can pass a string (if argument just one)
# message_prompt = prompt_template.invoke("Italia")

# prompt_template.input_schema

message_prompt

ChatPromptValue(messages=[HumanMessage(content='What is capital city of Italia?')])

In [2]:
message_prompt.to_messages() # => this is prompt ready to be passed into a language model

[HumanMessage(content='What is capital city of Italia?')]

Let's create a `PromptTemplate` that will generate a list of messages. It will accept two arguments:

- `language`: The target language of translation
- `text`: The tex to be translated

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Translate the following into {language}:"),
        ("user", "{text}"),
    ]
)   
message_prompt = prompt_template.invoke({"language": "italian", "text": "hi"})

message_prompt.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

## 2 - Alternative Ways To Build Prompt Template

The `ChatPromptTemplate` in LangChain can accept various formats for messages. Below are examples of each supported format:

### 1. `BaseMessagePromptTemplate`

**Example**:
```python
from langchain_core.prompts import BaseMessagePromptTemplate

# Define a custom message prompt template
custom_message_template = BaseMessagePromptTemplate(template="Hello, {user_name}! How can I assist you today?")

# Use this in ChatPromptTemplate
chat_prompt = ChatPromptTemplate(messages=[custom_message_template])
```

### 2. `BaseMessage`

**Example**:
```python
from langchain_core.prompts import BaseMessage

# Define a basic message
basic_message = BaseMessage(content="Please provide your feedback.")

# Use this in ChatPromptTemplate
chat_prompt = ChatPromptTemplate(messages=[basic_message])
```

### 3. 2-tuple of `(message type, template)`

**Example**:
```python
# Define a list of message types and templates
messages = [
    ("system", "You are a helpful assistant."),
    ("user", "I need information on {topic}.")
]

# Use this in ChatPromptTemplate
chat_prompt = ChatPromptTemplate(messages=messages)
```

### 4. 2-tuple of `(message class, template)`

**Example**:
```python
from langchain_core.prompts import HumanMessage, SystemMessage

# Define message classes and templates
messages = [
    (SystemMessage, "You are a knowledgeable advisor."),
    (HumanMessage, "I want to know more about {subject}.")
]

# Use this in ChatPromptTemplate
chat_prompt = ChatPromptTemplate(messages=messages)
```

These formats allow flexibility in how you define and use messages within `ChatPromptTemplate`, accommodating different needs and scenarios in your chatbot or conversational AI implementations.

See [ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain-core-prompts-chat-chatprompttemplate) for further

In [13]:
from langchain_core.prompts import HumanMessage, SystemMessage

prompt_template = ChatPromptTemplate(
    {
        SystemMessage: "Translate the following into {language}:",
        HumanMessage: "{text}",
    }
)   
message_prompt = prompt_template.invoke({"language": "italian", "text": "hi"})

message_prompt

ChatPromptValue(messages=[HumanMessage(content='system'), HumanMessage(content='user')])

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

model.invoke(message_prompt)

# model.invoke(message_prompt.to_messages)

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 15, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-054fbaec-d30a-4a6f-ab42-3d3f808db375-0', usage_metadata={'input_tokens': 15, 'output_tokens': 9, 'total_tokens': 24})

## `MessagePlaceholder`

MessagePlaceholder

In [10]:
from langchain_core.prompts import MessagesPlaceholder

# prompt = MessagesPlaceholder("history")
# prompt.format_messages() # raises KeyError

prompt = MessagesPlaceholder("history", optional=True)
prompt.format_messages() # returns empty list []

prompt.format_messages(
    history=[
        ("system", "You are an AI assistant."),
        ("human", "Hello!"),
    ]
)

[SystemMessage(content='You are an AI assistant.'),
 HumanMessage(content='Hello!')]

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ]
)

prompt.invoke(
   {
       "history": [("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
       "question": "now multiply that by 4"
   }
)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content="what's 5 + 2"), AIMessage(content='5 + 2 is 7'), HumanMessage(content='now multiply that by 4')])

## Output Parser

One useful and frequently used concept in langchain is output parser. It allows you to parse a AI response object into a desired format, for example string. See code below:

In [16]:
from langchain_core.output_parsers import StrOutputParser  
from langchain_core.messages import AIMessage

# typical AI respond
message = AIMessage(content="Hi Jim. How can I assist you today?")

parser = StrOutputParser()

parser.invoke(message)

'Hi Jim. How can I assist you today?'

We can also format output to different format, such as `JsonOutputParser`, `YamlOututParser`, or `CustomOutputParser`